In [4]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from spark_util import get_parser


from get_missing_articles import *
import os
import codecs
import networkx as nx
from collections import Counter
from pprint import pprint
from ConfigParser import SafeConfigParser
import pandas as pd

ImportError: No module named util

In [4]:
s = 'en'
t = 'fr'

cp = SafeConfigParser()
cp.read('/home/ellery/translation-recs-app/translation-recs.ini')

delim = '\t'

Got Wikidata Links


In [ ]:
wd_languages = set([s, t, r])
rd_languages = set([s, t, r, 'wikidata'])
ill_languages_from = set([s, t, r])
ill_languages_to = set([s, t, r])

G = create_graph(sc, cp, delim, wd_languages, rd_languages, ill_languages_from, ill_languages_to)


In [8]:
cc = nx.connected_component_subgraphs (G)
missing_items = {}
for i, g in enumerate(cc):
    missing_items.update(is_subgraph_missing_target_item(g, s, t, delim))

Got 4869687 missing items


In [ ]:
missing_items_df = pd.DataFrame(missing_items.items())
missing_items_df.columns = ['id', 'title']
missing_items_df = missing_items_df[missing_items_df['title'].apply(lambda x: (':' not in x) and (not x.startswith('List')))]

In [34]:
pageviews = sc.textFile(cp.get('general', 'pageviews'))\
    .map(lambda x: x.split('\t'))\
    .filter(lambda x: x[1] == s)\
    .map(lambda x: (x[0], int(x[3]))).collect()
pageviews_df = pd.DataFrame(pageviews)
pageviews_df.columns = ['id', 'n']

In [40]:
missing_items_df = missing_items_df.merge(pageviews_df, on='id')
missing_items_df = missing_items_df.sort('n', ascending = False)
fname = os.path.join(cp.get('general', 'local_data_dir'), exp_dir, cp.get('missing', 'missing_items'))
missing_items_df.to_csv(fname, sep='\t', encoding='utf8', index = False, header = False) 